In [ ]:
# Install dependencies for this example
# Note: This does not include itkwidgets, itself
import sys
!{sys.executable} -m pip install --upgrade itk-minimalpathextraction

In [1]:
from pathlib import Path

import itk
import numpy as np

from itkwidgets import view

In [2]:
DATA_DIR = Path('../test/Input')

In [3]:
dsa = itk.imread(str(DATA_DIR / 'Real-DSA-01.jpg'))
view(dsa, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC2; proxy …

In [4]:
dsa_speed = itk.imread(str(DATA_DIR / 'Real-DSA-01-Speed-02.mhd'))
view(dsa_speed, ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…

In [5]:
path_file = str(DATA_DIR / 'Real-DSA-01.path')
!cat {path_file}

Path: [394.00, 203.00] [388.00, 229.00] [356.00, 217.00] [289.00, 252.00] [194.00, 309.00] [162.00, 318.00] [232.00, 490.00]


In [6]:
PointType = itk.Point[itk.D,2]
PathInformationType = itk.SpeedFunctionPathInformation[PointType]

In [7]:
path_information = PathInformationType.New()
with open(path_file, 'r') as fp:
    for line in fp:
        line = line.replace("Path: ", "")
        line = line.replace("[", "").strip()
        points = line.split(']')[:-1]
        way_points = np.empty((0, 2))
        for index, point in enumerate(points):
            point_float = PointType()
            point_float[0] = float(point.split(',')[0])
            point_float[1] = float(point.split(',')[1])
            if index == 0:
                start_point = np.array(point_float).reshape(1,2)
                path_information.SetStartPoint(point_float)
            elif index == len(points)-1:
                end_point = np.array(point_float).reshape(1,2)
                path_information.SetEndPoint(point_float)
            else:
                way_points = np.vstack((way_points, np.array(point_float).reshape(1,2)))
                path_information.AddWayPoint(point_float)

In [8]:
view(dsa_speed,
     point_sets=[start_point, way_points, end_point],
     point_set_colors=[(1.0,0.0,0.0), (1.0,0.0,0.5), (1.0,0.0,0.8)],
     ui_collapsed=True)

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[1. , 0. , 0. ],
       [1. , 0. , 0.5],
…

In [9]:
interpolator = itk.LinearInterpolateImageFunction.New(dsa_speed)
interpolator.SetInputImage(dsa_speed)

In [10]:
cost_function = itk.SingleImageCostFunction[type(dsa_speed)].New(interpolator=interpolator)
cost_function.SetInterpolator(interpolator)

In [11]:
spacing = list(dsa_speed.GetSpacing())
min_spacing = min(spacing)
step_length_factor = 0.25
step_length_relax = 0.5
number_of_iterations = 4000

optimizer = itk.RegularStepGradientDescentOptimizer.New(
    number_of_iterations=number_of_iterations,
    maximum_step_length=1.0*step_length_factor*min_spacing,
    minimum_step_length=0.5*step_length_factor*min_spacing,
    relaxation_factor=step_length_relax)

In [12]:
termination_value = 3.0
path_filter = itk.SpeedFunctionToPathFilter.New(dsa_speed,
                                               cost_function=cost_function,
                                               optimizer=optimizer,
                                               termination_value=termination_value)
path_filter.SetInput(dsa_speed)

In [13]:
path_filter.AddPathInformation(path_information)

In [14]:
%time path_filter.Update()

CPU times: user 64.8 ms, sys: 0 ns, total: 64.8 ms
Wall time: 64.2 ms


In [15]:
path = path_filter.GetOutput(0)
path.GetVertexList().Size()

2232

In [16]:
view(dsa, geometries=path, ui_collapsed=True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…